# Fetch Rewards Coding Exercise - Data Analyst

## First: Data Loading

Loaded the unstructured Users & Brands JSON data. A relational data model diagram has been place in the repository (ER_diagram.pdf) 

In [204]:
from pandas import json_normalize
import json
import pandas as pd
import datetime

usersList = []
with open("E:\\Fetch\\users.json") as f:
    for jsonObj in f:
        usersDict = json.loads(jsonObj)
        usersList.append(usersDict)
df_users = json_normalize(usersList)

brandsList = []
with open("E:\\Fetch\\brands.json") as f:
    for jsonObj in f:
        brandsDict = json.loads(jsonObj)
        brandsList.append(brandsDict)
df_brands = json_normalize(brandsList)


Loaded the receipts data into a "df_receipts" dataframe except for the "rewardsReceiptItemList" field. This field has a receipts list for each item, so i have loaded those details unto a new dataframe called "df_itemslist".

In [205]:
df_receipts = [json.loads(k) for k in open('E:\\Fetch\\receipts.json','r')]
#Format receipts data
temp = []
temp1 = []
for b in range(0,len(df_receipts)):
    k = {}
    for i,j in df_receipts[b].items():
        if i == '_id':
            p = j['$oid']
            j=p
            tempid = p
            k[i] = j
        if i in ('createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate'):
            p = int(j['$date'])
            j=p/1000
            j=datetime.datetime.fromtimestamp(j)
            j=j.strftime('%Y-%m-%d')
            k[i] = j
        if i == 'rewardsReceiptItemList':
            for c in range(0,len(j)):
                m = {}
                for a,b in j[c].items():
                    m[a] = b
                    m['_id'] = tempid
                temp1.append(m)
        else:
            k[i]=j
    temp.append(k)
df_receipts = pd.DataFrame(temp)
df_itemslist = pd.DataFrame(temp1)

Renameing the columns in users and brands dataframes.

In [206]:
df_users = df_users.rename(columns={'_id.$oid' : '_id', 'createdDate.$date' : 'createdDate', 
                                    'lastLogin.$date' : 'lastLogin'})
df_brands = df_brands.rename(columns={'_id.$oid' : '_id', 'cpg.$id.$oid' : 'cpg_id', 'cpg.$ref' : 'cpg_ref'})

The date columns in users dataframe is converted from 13 digit date format to datetime format

In [207]:
df_users['createdDate'] = df_users['createdDate'].apply(lambda timestamp : 
                                                                    datetime.datetime.fromtimestamp(int(timestamp)/1000))
df_users['createdDate'] = pd.to_datetime(df_users['createdDate'])
df_users['lastLogin'] = df_users[~df_users['lastLogin'].isnull()].loc[:,'lastLogin'].apply(lambda timestamp : 
                                                                    datetime.datetime.fromtimestamp(int(timestamp)/1000))
df_users['lastLogin'] = pd.to_datetime(df_users['lastLogin'])

## Receipts Data

In [209]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,500.0,2021-01-02,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,150.0,2021-01-02,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,NaN,2021-01-03,NaN,5,2021-01-02,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,5.0,2021-01-02,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,5.0,2021-01-02,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


## Items List

In [210]:
df_itemslist.head()

,barcode,_id,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,5ff1e1eb0a720f0523000575,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,5ff1e1bb0a720f052300056b,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,5ff1e1bb0a720f052300056b,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,5ff1e1f10a720f052300057a,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,5ff1e1ee0a7214ada100056f,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Users Data

In [211]:
df_users.head()

,active,role,signUpSource,state,_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 10:25:30.554,2021-01-03 10:25:30.597
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04.800,2021-01-03 10:25:37.858


## Brands Data

In [212]:
df_brands.head()

,barcode,category,categoryCode,name,topBrand,_id,cpg_id,cpg_ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


## Second: Queries to answer predetermined business questions

Question : When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [216]:
mysql("select avg(totalspent) as Average_Spent, rewardsReceiptStatus as Rewards_Receipt_Status from df_receipts where rewardsReceiptStatus = 'ACCEPTED'")

,Average_Spent,Rewards_Receipt_Status
0,None,None


In [217]:
mysql("select avg(totalspent) as Average_Spent, rewardsReceiptStatus as Rewards_Receipt_Status from df_receipts where rewardsReceiptStatus = 'REJECTED' ")

,Average_Spent,Rewards_Receipt_Status
0,23.326056,REJECTED


After executing the above 2 queries we can see that there are no records in receipts data with 'rewardsReceiptStatus’ as 'ACCEPTED'. There are records with 'REJECTED' status which has an average spend of 23.326056

Question: When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [218]:
mysql("select sum(purchasedItemCount) as 'Total number of items purchased', rewardsReceiptStatus as 'Status' from df_receipts group by rewardsReceiptStatus ")

,Total number of items purchased,Status
0,8184.0,FINISHED
1,1014.0,FLAGGED
2,NaN,PENDING
3,173.0,REJECTED
4,NaN,SUBMITTED


As we have already seen in previous question, there are no records with 'ACCEPTED' status for reqards receipt we did not get the count of total number of items prchased in above query. For the count of items purchased for 'REJECTED' status we have 173 items.

## Third: Evaluate Data Quality Issues in the Data Provided

More than half of the records in Users table is redundent with only 212 unique user details availabe out of 495 availabe records.

In [228]:
len_users = len(df_users)
print("Number of rows in Users DataFrame:",len_users)
dup_users = df_users.duplicated().sum()
print("Number of duplicate rows in Users DataFrame:",dup_users)
unique_users = len_users - dup_users
print("Number of Unique unsers in our database:",unique_users)

Number of rows in Users DataFrame: 495
Number of duplicate rows in Users DataFrame: 283
Number of Unique unsers in our database: 212


There are 117 user_id's in receipts database that are not availbe  in users database

In [231]:
df_missing_users = mysql("select * from df_receipts a left join df_users b on a.userId = b._id where b._id is null")
print("Count of users whose details are missing:",len(pd.unique(df_missing_users['userId'])))

Count of users whose details are missing: 117


There are 71 users in user database that does not have a receipt availabe. We will have to discuss with business stakeholders and find out the reason why these users are not having receipts in our database.

In [234]:
df_missing_receipts = mysql("select a._id from df_users a left join df_receipts b on a._id = b.userId where b.userId is null")
print("Count of users who does not have receipts availble:",len(pd.unique(df_missing_receipts['_id'])))

Count of users who does not have receipts availble: 71


Almost half of the data is missing for some of the below given fields. This missing data could be of concern in our analysis. 

In [237]:
for k,v in df_receipts.isnull().mean().items():
    if v > 0:
        print(k,":",v*100)

bonusPointsEarned : 51.385165326184094
bonusPointsEarnedReason : 51.385165326184094
finishedDate : 49.240393208221626
pointsAwardedDate : 52.01072386058981
pointsEarned : 45.57640750670242
purchaseDate : 40.03574620196604
purchasedItemCount : 43.25290437890974
totalSpent : 38.8739946380697


We do not have details on items bought for 440 receipts since the field "rewardsReceiptItemList" is not availabe in receipts database

In [241]:
df_missing_itemslist = mysql("select DISTINCT a._id from df_receipts a left join df_itemslist b on a._id = b._id where b._id is null")
print("Count of receipts that does not have any items listed:",len(df_missing_itemslist))

Count of receipts that does not have any items listed: 440


More than 50% of the data is missing for topBrand & catagoryCode fields.

In [243]:
for k,v in df_brands.isnull().mean().items():
    if v > 0:
        print(k,":",v*100)

category : 13.281919451585262
categoryCode : 55.69837189374465
topBrand : 52.44215938303341
brandCode : 20.051413881748072
